# Experiment 3: TFIDF + LDA + SVM

In [1]:
# Load for Jupyter Notebook
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('/home/elenaruiz/Documents/FNC')
import pandas as pd 
import numpy as np 
from src.utils import io

from src.fake_news_detector.core.encoders import bow as b
from src.fake_news_detector.core.encoders import tfidf as t

## Import `raw_dataset.json`

In [2]:
articles = io.read_json_file('/home/elenaruiz/Documents/FNC/src/data/dataset_content.json')
df = pd.DataFrame(data=articles['articles']) # Put in pandas dataframe
df.head()

,adjective_words,all_joined,all_word,common_noun_words,conjunction_words,fake,negative_words,noun_phrases_words,positive_words,title_adjective_words,title_common_noun_words,title_conjunction_words,title_negative_words,title_noun_phrases_words,title_positive_words
0,"[vegetarian, international, human, middle, ope...",The Thai police have clarified to the middle A...,"[they, find, a, corpse, in, a, vegetarian, res...","[corpse, restaurant, Bangkok, find, speculate,...","[but, and]",True,"[corpse, kill, fire, complain, deny, victim, v...","[corpse, vegetarian restaurant, Bangkok, find,...",[like],[vegetarian],"[corpse, restaurant, Bangkok]",[],[corpse],"[corpse, vegetarian restaurant, Bangkok]",[]
1,"[political, Swiss, legal, eventual, underline,...","The Swiss government has said Tuesday that ""a ...","[switzerland, warn, that, it, will, not, autho...","[switzerland, warn, extradition, crimes, A, sp...","[or, and, but]",True,"[no, stress, offences, offences, accuse, argue...","[switzerland warn, extradition, political crim...","[Justice, like, Supreme, fair, perfectly]",[political],"[switzerland, warn, extradition, crimes]",[],[],"[switzerland warn, extradition, political crimes]",[]
2,"[navarre, educational, several, great, last, f...","The Government of Navarra, within the Skolae p...","[navarre, censor, Songs, of, Amaral, Shakira, ...","[censor, Songs, Amaral, Shakira, song, Madman,...","[and, but, or]",True,"[censor, violence, ban, bad, fight, shit]","[navarre censor Songs, Amaral Shakira, song, M...","[promote, great, hope, promote, like, affectio...",[navarre],"[censor, Songs, Amaral, Shakira, song, Madman,...",[and],[censor],"[navarre censor Songs, Amaral Shakira, song, M...",[]
3,"[Spanish, i, social, pose, many, hard, whole, ...",Carmen Jiménez told her family and friends 28 ...,"[a, woman, pretend, to, be, blind, for, years,...","[woman, pretend, years, people, truth, Carmen,...",[and],True,"[blind, blind, injury, blind, avoid]","[woman pretend, years, people, truth, Carmen J...","[greet, truth, friends, truth, greet, justice]",[],"[woman, pretend, years, people]",[],[blind],"[woman pretend, years, people]",[greet]
4,"[last, past, despicable, strong, superior, ass...","Lewis Williams, a worker at an engineering fir...","[arrested, for, ejaculate, at, her, boss, coff...","[ejaculate, boss, coffee, years, action, discu...","[four, and]",True,"[arrested, detain, arrest, prison, abuse]","[ejaculate, boss coffee, years, action, discus...","[United, strong, superior, proud]",[last],"[ejaculate, boss, coffee, years]",[four],[arrested],"[ejaculate, boss coffee, years]",[]


In [3]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

## Dictionary creation and word vectorization with TF-IDF

In [4]:
from src.fake_news_detector.core.classificators import latent_analysis as la
X_train = df_train['all_word'].values
Y_train = df_train['fake'].values
X_test = df_test['all_word'].values
Y_test = df_test['fake'].values

In [5]:
import gensim
from gensim import corpora, models

dictionary = gensim.corpora.Dictionary(X_train)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in X_train]
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

## Topic modeling transformation with LDA

In [8]:
import matplotlib.pyplot as plt

def get_optimal(lda_model, corpus, encoded_data, dictionary, min_topic, max_topic):
    idt_values = range(min_topic, max_topic)
    perp_values = []
    cohe_values = []
    for topic in idt_values:
        perp_values.append(lda_model.log_perplexity(encoded_data))
        
        coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus, dictionary=dictionary, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        cohe_values.append(coherence_lda)
        
    plt.figure(1)
    plt.subplot(211)
    plt.plot(idt_values, perp_values, 'ro')

    plt.subplot(212)
    plt.plot(idt_values, cohe_values, 'bo')
    plt.show()

In [6]:
# With BOW
bow_lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
get_optimal(lda_model, corpus, encoded_data, dictionary, min_topic, max_topic)

/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


Topic: 0 
Words: 0.046*"his" + 0.044*"most" + 0.043*"place" + 0.038*"where" + 0.029*"take" + 0.027*"say" + 0.026*"after" + 0.026*"make" + 0.020*"because" + 0.020*"come"
Topic: 1 
Words: 0.042*"his" + 0.037*"years" + 0.031*"all" + 0.028*"police" + 0.020*"when" + 0.020*"find" + 0.019*"against" + 0.019*"I" + 0.019*"other" + 0.018*"him"
Topic: 2 
Words: 0.049*"use" + 0.029*"can" + 0.028*"Spanish" + 0.026*"already" + 0.024*"their" + 0.022*"when" + 0.020*"out" + 0.019*"go" + 0.018*"than" + 0.017*"our"
Topic: 3 
Words: 0.029*"say" + 0.027*"their" + 0.027*"year" + 0.024*"make" + 0.022*"first" + 0.021*"Spain" + 0.019*"while" + 0.019*"if" + 0.019*"new" + 0.018*"between"
Topic: 4 
Words: 0.037*"his" + 0.031*"de" + 0.025*"what" + 0.023*"say" + 0.022*"there" + 0.021*"I" + 0.021*"we" + 0.020*"all" + 0.020*"time" + 0.018*"Spanish"
Topic: 5 
Words: 0.041*"euros" + 0.036*"after" + 0.029*"government" + 0.028*"work" + 0.028*"people" + 0.026*"his" + 0.024*"these" + 0.023*"end" + 0.023*"if" + 0.021*"years"

In [7]:
# With TF-IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
la.print_top_words(lda_model_tfidf)

/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


Topic: 0 
Words: 0.034*"use" + 0.027*"Spanish" + 0.022*"would" + 0.021*"act" + 0.019*"report" + 0.018*"people" + 0.018*"find" + 0.017*"try" + 0.017*"Spain" + 0.016*"social"
Topic: 1 
Words: 0.034*"you" + 0.028*"their" + 0.022*"European" + 0.021*"out" + 0.017*"state" + 0.017*"most" + 0.017*"police" + 0.017*"our" + 0.016*"government" + 0.016*"any"
Topic: 2 
Words: 0.022*"her" + 0.020*"Spain" + 0.019*"only" + 0.019*"you" + 0.016*"place" + 0.016*"say" + 0.016*"case" + 0.016*"de" + 0.015*"his" + 0.015*"would"
Topic: 3 
Words: 0.037*"Spanish" + 0.027*"both" + 0.019*"his" + 0.019*"between" + 0.018*"them" + 0.018*"lead" + 0.017*"European" + 0.017*"although" + 0.017*"last" + 0.016*"these"
Topic: 4 
Words: 0.025*"euros" + 0.023*"his" + 0.017*"most" + 0.017*"than" + 0.017*"three" + 0.017*"against" + 0.017*"him" + 0.016*"their" + 0.016*"between" + 0.015*"up"
Topic: 5 
Words: 0.036*"new" + 0.030*"first" + 0.027*"can" + 0.027*"country" + 0.021*"our" + 0.019*"another" + 0.018*"must" + 0.018*"believe"